# Recurrent Neural Networks

RRNs are designed to learn from sequential data. 
It performs the same task for every element of a sequence.


RNNs are used in language modeling, ie predicting what word come next for a given sequence of words (sentence).

RNN should work on sequences of arbitrary lengths of input elements.

So that, RNNs model have connections backward. It has a memory of current state that has been calculated so far.


We can consider RNNs create a new layer of neurons for each input, but each layer shares a common set of weights. 

Let consider;

- $x_t$ as the input at time $t$
- $h_t$ as the hidden state at time $t$
- $o_t$ as the output at time $t$

Imagine we are going to train a sequence of words, where each word is in one-hot encoded form. 

Then $x_t \in {\rm I\!R}^{v x 1} $, where $v$ is the size of the vocabulary

The hidden layer can be defined with any length of choice, $n$.
Therefore, weight metric connecting input layar to hidden layer will be $W_{xh} \in {\rm I\!R}^{n x v}$

The hidden layer $h_t$ is connected to next hidden layer $h_{t+1}$ with weight metric $W_{hh} \in {\rm I\!R}^{n x n}$

$h_t$ is represented as;

$
h_t = f(W_{hh} h_{t-1} + W_{xh} x_t) \text{, where $f$ is a non-linear activation funtion, e.g., tanh or ReLU}
$

The connection from hidden layer to the output layer is just like connection from a fully connected layer to the output layer as in CNN.

The predicted output vector $o_t \in {\rm I\!R}^{v x 1} $ is connected from hiden layer with weight metric $W_{ho} \in {\rm I\!R}^{n x v}$ 

$o_t$ can be represented as; 

$
o_t = g(W_{ho} h_{t} + b_o) \text{, where $g$ is softmax function and $b \in {\rm I\!R}^{n x 1}$ is a bias vector }
$


The cost at each time step is the negative log cost over all the vocabulary size $v$.

$
cost(y_t, o_t) = cross-entropy(y_t, o_t) = - \sum_{i=1}^{v}y_t log o_{t}
$


## LSTM Recurrent Neural Networks

LSTM is a special version of RNN that can learn past information in a sequance.

Standard RNN cannot learn correlation between distance element in a sequance.

are special versions
of RNNs that can learn distant dependencies between words in a sentence. 

At sequance step $t$, with the input $x_i$ and the previous step's hidden states $h_{t-1}$;
- decide what to forget from cell state $C_{t-1}$ throught the `forget layer`, which outputs $f_t = g(W_f x_t + W_{fh} h_{t-1})$, where $g$ is sigmoid activiation.
- next `input layer` decide which new information should be updated in the cell state, which output $i_t=g(W_i x_t + W_{ih} h_{t-1})$, where $g$ is sigmoid activiation.
- the, a candidate set of new values, $\overline{C}_t$ is created for the cell state using $\overline{C}_t=tanh(W_c x_t + W_{ch} h_{t-1})$
- then the cell state $C_t$ is updated as $C_t = f_t * C_{t-1} + i_t * \overline{C}_t$, where `*` represent element wise multiplication.
- the `output layer` determine which to output from cell state as it contains a lot of information, $o_t = g(W_{o} x_t + W_{oh} h_{t-1})$, where $g$ is sigmoid activiation.
- finally the hidden state $h_t$ is updated with cell state as $h_t = o_t * tanh(C_t)$





## Next-Word Prediction with LSTM RNN

The following passage from `Alice in Wonderland` is used to predict the next word using LSTM.
(The example and code is taken from the book, Pro Deep Learning with TensorFlow by Santanu Pattanayak)


In [115]:
def print_file(fname):
    with open(fname) as f:
        print(f.readlines())
        
print_file('AliceInWonderland.txt')

["' You can't think how glad I am to see you again , you dear old thing ! ' said the Duchess , as she tucked her arm affectionately into Alice's , and they walked off together . Alice was very glad to find her in such a pleasant temper , and thought to herself that perhaps it was only the pepper that had made her so savage when they met in the kitchen . ' When I'm a Duchess , ' she said to herself , ( not in a very hopeful tone though ) , ' I won't have any pepper in my kitchen at all . Soup does very well without \xe2\x80\x94 Maybe it's always pepper that makes people hot-tempered , ' she went on , very much pleased at having found out a new kind of rule , ' and vinegar that makes them sour \xe2\x80\x94 and camomile that makes them bitter \xe2\x80\x94 and \xe2\x80\x94 and barley-sugar and such things that make children sweet-tempered . I only wish people knew that : then they wouldn't be so stingy about it , you know \xe2\x80\x94 'She had quite forgotten the Duchess by this time , and

In [165]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

tf.reset_default_graph()

A sequence of 3 words is used as the input, and subsequent word is taken as the output.

In [167]:
learning_rate = 0.001
training_iters = 50000
display_step = 500
n_input = 3

n_hidden = 512

In [168]:
# Function to read and process the input file
def read_data(fname):
    with open(fname) as f:
        data = f.readlines()
    data = [x.strip() for x in data]
    data = [data[i].lower().split() for i in range(len(data))]
    data = np.array(data)
    data = np.reshape(data, [-1, ])
    return data

In [169]:
# Function to build dictionary and reverse dictionary of words.
def build_dataset(train_data):
    count = collections.Counter(train_data).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [170]:
train_file = 'AliceInWonderland.txt'
train_data = read_data(train_file)
dictionary, reverse_dictionary = build_dataset(train_data)

In [171]:
# Place holder for Mini-batch input output
x = tf.placeholder("float", [None, n_input, vocab_size])
y = tf.placeholder("float", [None, vocab_size])

In [172]:
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

### RNN Model

A two-layered LSTM model is used.




In [173]:
# Forward pass for the recurrent neural network
def RNN(x, weights, biases):
    x = tf.unstack(x, n_input, 1)
    # 2 layered LSTM Definition
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [174]:
pred = RNN(x, weights, biases)

In [175]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

In [176]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

At each iteration, 3 sequence of input words and its subsequent word as the output are selected randomly.


In [177]:
# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    
    while step < training_iters:
        if offset > (len(train_data)-end_offset):
            offset = random.randint(0, n_input+1)
            
        symbols_in_keys = [ input_one_hot(dictionary[ str(train_data[i])]) for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input,vocab_size])
        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])
        
        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        
        loss_total += loss
        acc_total += acc
        
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            
            print("%s - Actual word:[%s] vs Predicted word:[%s]" % (symbols_in,symbols_out,symbols_out_pred))
            
        step += 1
        offset += (n_input+1)
    print("TrainingCompleted!")
    
# Feed a 3-word sentence and let the model predict the next 28 words
    sentence = 'i only wish'
    words = sentence.split(' ')
    try:
        symbols_in_keys = [ input_one_hot(dictionary[ str(train_data[i])]) for i in
        range(offset, offset+n_input) ]
        for i in range(28):
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_input,vocab_size])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = symbols_in_keys[1:]
            symbols_in_keys.append(input_one_hot(onehot_pred_index))
        print "Complete sentence follows!"
        print(sentence)
    except:
        print("Error while processing the sentence to be completed")

            

Iter= 500, Average Loss= 4.537830, Average Accuracy= 11.00%
['people', 'knew', 'that'] - Actual word:[:] vs Predicted word:[:]
Iter= 1000, Average Loss= 3.685317, Average Accuracy= 23.40%
['she', 'did', 'not'] - Actual word:[like] vs Predicted word:[like]
Iter= 1500, Average Loss= 0.781747, Average Accuracy= 77.00%
['that', 'makes', 'them'] - Actual word:[bitter] vs Predicted word:[bitter]
Iter= 2000, Average Loss= 0.844376, Average Accuracy= 82.60%
['very', 'ugly', ';'] - Actual word:[and] vs Predicted word:[and]
Iter= 2500, Average Loss= 1.751117, Average Accuracy= 61.00%
['pleased', 'at', 'having'] - Actual word:[found] vs Predicted word:[found]
Iter= 3000, Average Loss= 0.572721, Average Accuracy= 86.20%
['keeping', 'so', 'close'] - Actual word:[to] vs Predicted word:[to]
Iter= 3500, Average Loss= 0.602516, Average Accuracy= 85.40%
['that', 'makes', 'people'] - Actual word:[hot-tempered] vs Predicted word:[forget]
Iter= 4000, Average Loss= 0.537070, Average Accuracy= 88.00%
["'", '

Iter= 32000, Average Loss= 0.000043, Average Accuracy= 100.00%
['side', 'as', 'she'] - Actual word:[spoke] vs Predicted word:[spoke]
Iter= 32500, Average Loss= 0.138463, Average Accuracy= 98.60%
['well', 'without', '\xe2\x80\x94'] - Actual word:[maybe] vs Predicted word:[maybe]
Iter= 33000, Average Loss= 0.082652, Average Accuracy= 98.60%
["'", 'and', 'she'] - Actual word:[squeezed] vs Predicted word:[squeezed]
Iter= 33500, Average Loss= 0.100275, Average Accuracy= 97.80%
['kitchen', 'at', 'all'] - Actual word:[.] vs Predicted word:[.]
Iter= 34000, Average Loss= 0.059269, Average Accuracy= 98.40%
['if', 'only', 'you'] - Actual word:[can] vs Predicted word:[can]
Iter= 34500, Average Loss= 0.054746, Average Accuracy= 99.00%
['not', 'in', 'a'] - Actual word:[very] vs Predicted word:[very]
Iter= 35000, Average Loss= 0.060881, Average Accuracy= 99.40%
['.', "'", 'tut'] - Actual word:[,] vs Predicted word:[,]
Iter= 35500, Average Loss= 0.058720, Average Accuracy= 99.40%
['the', 'kitchen', '.